##### Installing the required libraries

In [ ]:
#%pip install pinecone-client llama-index-vector-stores-pinecone

##### Loading dotenv file

In [1]:
from dotenv import load_dotenv
import os

# for loading GROQ_API_KEY and PINECONE_API_KEY
load_dotenv()

True

##### Importing libraries

In [2]:
from llama_index.core import (
    Settings, # setting the configuration of LLM to be used
    VectorStoreIndex, # default vector store indexing used in llamaindex
    SimpleDirectoryReader, # reading directory containing data files
    StorageContext
)
from pinecone import Pinecone, ServerlessSpec
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding # using HuggingFace embeddings
from llama_index.llms.groq import Groq # using Groq LLM 
import time 

##### Initializing APIs and Pinecone client

In [3]:
GROQ_API_KEY = os.environ["GROQ_API_KEY"]
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
pc = Pinecone(api_key = PINECONE_API_KEY)

##### Setting LLM configuration for LlamaIndex

In [4]:
llm = Groq(model = "llama3-8b-8192")
embed_model = HuggingFaceEmbedding(model_name = "sentence-transformers/all-mpnet-base-v2")

Settings.llm = llm
Settings.embed_model = embed_model

##### Pinecone index creation

In [5]:
# setting index name
index_name = "llama3-groq-pinecone"

# checking existing indexes
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# creating an index if it doesn't exists in Pinecone vector db
if index_name not in existing_indexes:
    pc.create_index(
        name = index_name, 
        dimension = 768, 
        metric = "cosine",
        spec = ServerlessSpec(
            cloud = "aws",
            region = "us-east-1"
        )
    )
    
# connecting to the created index
index = pc.Index(index_name)
time.sleep(1)

# describing index statistics
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 2}},
 'total_vector_count': 2}

##### Loading data from directory using LlamaIndex

In [6]:
documents = SimpleDirectoryReader("data_files").load_data()

##### Upserting data in Pinecone vector db

In [7]:
vector_store = PineconeVectorStore(pinecone_index = index)
storage_context = StorageContext.from_defaults(vector_store = vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context = storage_context
)

Upserted vectors:   0%|          | 0/2 [00:00<?, ?it/s]

##### Executing query using LlamaIndex and Pinecone vector db

In [8]:
query_engine = index.as_query_engine()
response = query_engine.query("Who won the 2024 presidential elections in USA?")
print(response)

Donald Trump and JD Vance won the 2024 presidential elections in the United States.


In [ ]:
""